In [13]:
import os
sys.path.append(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 8\timeseires')

In [14]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score
from timeseires.utils.to_split import to_split
from timeseires.utils.multivariate_multi_step import multivariate_multi_step
from timeseires.utils.multivariate_single_step import multivariate_single_step
from timeseires.utils.univariate_multi_step import univariate_multi_step
from timeseires.utils.univariate_single_step import univariate_single_step
from timeseires.utils.CosineAnnealingLRS import CosineAnnealingLRS
from timeseires.callbacks.EpochCheckpoint import EpochCheckpoint
from tensorflow.keras.callbacks import ModelCheckpoint
from timeseires.callbacks.TrainingMonitor import TrainingMonitor
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import LSTM, Bidirectional, Add
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv1D,TimeDistributed
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,MaxPooling1D,Concatenate,AveragePooling1D, GlobalMaxPooling1D, Input
from tensorflow.keras.models import Sequential,Model
import pandas as pd
import time, pickle
import numpy as np
import tensorflow.keras.backend as K
import tensorflow
from tensorflow.keras.layers import Input, Reshape, Lambda
from tensorflow.keras.layers import Layer, Flatten, LeakyReLU, concatenate, Dense
from tensorflow.keras.regularizers import l2
import glob
import h5py
import matplotlib.pyplot as plt
from keras.callbacks import Callback

In [15]:
#lookback = 24
model = None
start_epoch = 0
time_steps=24
num_features=21

In [16]:
def CNN():
    input_data = Input(shape=(time_steps, num_features))
    x1 = Conv1D(16, 2, activation="relu")(input_data)
    x2 = Conv1D(16, 2, activation="relu")(x1)
    flatten = Flatten()(x2)
    output_data = Dense(1)(flatten)
    model = Model(input_data, output_data)
    return model

In [17]:
model1 = CNN()
model1.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 24, 21)]          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 23, 16)            688       
                                                                 
 conv1d_3 (Conv1D)           (None, 22, 16)            528       
                                                                 
 flatten_1 (Flatten)         (None, 352)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 353       
                                                                 
Total params: 1,569
Trainable params: 1,569
Non-trainable params: 0
_________________________________________________________________


In [18]:
tensorflow.keras.utils.plot_model(model1 )

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [19]:
checkpoints = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 8\\E1-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
OUTPUT_PATH = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 8'
FIG_PATH =  os.path.sep.join([OUTPUT_PATH,"\history.png"])
JSON_PATH = os.path.sep.join([OUTPUT_PATH,"\history.json"])

In [20]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]

In [21]:
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model =CNN()
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] compiling model...


In [24]:
import os
path_dataset ='C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 8\\AEP_hourly_2'
path_tr = os.path.join(path_dataset, 'AEP_train.csv')
df_tr = pd.read_csv(path_tr)
train_set = df_tr.iloc[:].values
path_v = os.path.join(path_dataset, 'AEP_validation.csv')
df_v = pd.read_csv(path_v)
validation_set = df_v.iloc[:].values 
path_te = os.path.join(path_dataset, 'AEP_test.csv')
df_te = pd.read_csv(path_te)
test_set = df_te.iloc[:].values 

path_scaler = os.path.join(path_dataset, 'AEP_Scaler.pkl')
scaler         = pickle.load(open(path_scaler, 'rb'))

train_set.shape, validation_set.shape, test_set.shape

((84907, 21), (24259, 21), (12130, 21))

In [25]:
time_steps=24
num_features=21

In [26]:
start = time.time()
train_X , train_y = univariate_multi_step(train_set, time_steps, target_col=0,target_len=1)
validation_X, validation_y = univariate_multi_step(validation_set, time_steps, target_col=0,target_len=1)
test_X, test_y = univariate_multi_step(test_set, time_steps, target_col=0,target_len=1)
print('Time Consumed', time.time()-start, "sec")

Time Consumed 5.220271348953247 sec


In [27]:
epochs = 60
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,verbose = verbose)

Epoch 1/60
2652/2653 [============================>.] - ETA: 0s - loss: 0.0426 - mae: 0.0426 - mape: 648.6614
Epoch 1: val_loss improved from inf to 0.02426, saving model to C:\Users\musta\Desktop\ML lab\lab 8\E1-cp-0001-loss0.02.h5
2653/2653 [==============================] - 35s 12ms/step - loss: 0.0426 - mae: 0.0426 - mape: 648.5253 - val_loss: 0.0243 - val_mae: 0.0243 - val_mape: 10.9787
Epoch 2/60
2652/2653 [============================>.] - ETA: 0s - loss: 0.0210 - mae: 0.0210 - mape: 587.7868
Epoch 2: val_loss improved from 0.02426 to 0.01631, saving model to C:\Users\musta\Desktop\ML lab\lab 8\E1-cp-0002-loss0.02.h5
2653/2653 [==============================] - 36s 13ms/step - loss: 0.0210 - mae: 0.0210 - mape: 587.6629 - val_loss: 0.0163 - val_mae: 0.0163 - val_mape: 7.4489
Epoch 3/60
2652/2653 [============================>.] - ETA: 0s - loss: 0.0163 - mae: 0.0163 - mape: 800.0812
Epoch 3: val_loss improved from 0.01631 to 0.01531, saving model to C:\Users\musta\Desktop\ML lab

Epoch 25/60
2648/2653 [============================>.] - ETA: 0s - loss: 0.0085 - mae: 0.0085 - mape: 271.4257
Epoch 25: val_loss improved from 0.00791 to 0.00760, saving model to C:\Users\musta\Desktop\ML lab\lab 8\E1-cp-0025-loss0.01.h5
2653/2653 [==============================] - 21s 8ms/step - loss: 0.0085 - mae: 0.0085 - mape: 270.9630 - val_loss: 0.0076 - val_mae: 0.0076 - val_mape: 3.7300
Epoch 26/60
2648/2653 [============================>.] - ETA: 0s - loss: 0.0085 - mae: 0.0085 - mape: 419.8245
Epoch 26: val_loss improved from 0.00760 to 0.00724, saving model to C:\Users\musta\Desktop\ML lab\lab 8\E1-cp-0026-loss0.01.h5
2653/2653 [==============================] - 23s 9ms/step - loss: 0.0085 - mae: 0.0085 - mape: 419.1065 - val_loss: 0.0072 - val_mae: 0.0072 - val_mape: 3.8858
Epoch 27/60
2649/2653 [============================>.] - ETA: 0s - loss: 0.0084 - mae: 0.0084 - mape: 376.6553
Epoch 27: val_loss did not improve from 0.00724
2653/2653 [==============================] 

Epoch 50/60
2647/2653 [============================>.] - ETA: 0s - loss: 0.0081 - mae: 0.0081 - mape: 572.4722
Epoch 50: val_loss did not improve from 0.00710
2653/2653 [==============================] - 19s 7ms/step - loss: 0.0081 - mae: 0.0081 - mape: 571.2766 - val_loss: 0.0075 - val_mae: 0.0075 - val_mape: 3.8432
Epoch 51/60
2647/2653 [============================>.] - ETA: 0s - loss: 0.0080 - mae: 0.0080 - mape: 392.2985
Epoch 51: val_loss did not improve from 0.00710
2653/2653 [==============================] - 18s 7ms/step - loss: 0.0080 - mae: 0.0080 - mape: 391.4815 - val_loss: 0.0078 - val_mae: 0.0078 - val_mape: 3.8407
Epoch 52/60
2647/2653 [============================>.] - ETA: 0s - loss: 0.0081 - mae: 0.0081 - mape: 645.0965
Epoch 52: val_loss did not improve from 0.00710
2653/2653 [==============================] - 20s 8ms/step - loss: 0.0081 - mae: 0.0081 - mape: 643.7507 - val_loss: 0.0079 - val_mae: 0.0079 - val_mape: 4.0727
Epoch 53/60
2647/2653 [====================

In [28]:

model = load_model(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 8\E1-cp-0059-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 8s 7ms/step
Mean Absolute Error (MAE): 107.37
Median Absolute Error (MedAE): 87.05
Mean Squared Error (MSE): 19766.47
Root Mean Squared Error (RMSE): 140.59
Mean Absolute Percentage Error (MAPE): 0.74 %
Median Absolute Percentage Error (MDAPE): 0.59 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


In [29]:
checkpoints = r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 8 E2-cp-{epoch:04d}-loss{val_loss:.2f}.h5'
model=rC:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 4\E1-cp-0059-loss0.01.h5'
start_epoch= 59

In [30]:
# construct the callback to save only the *best* model to disk
# based on the validation loss
EpochCheckpoint1 = ModelCheckpoint(checkpoints,
                             monitor="val_loss",
                             save_best_only=True, 
                             verbose=1)
TrainingMonitor1=TrainingMonitor(FIG_PATH, jsonPath=JSON_PATH, startAt=start_epoch)

# construct the set of callbacks
callbacks = [EpochCheckpoint1,TrainingMonitor1]
# if there is no specific model checkpoint supplied, then initialize
# the network and compile the model
if model is None:
    print("[INFO] compiling model...")
    model = PC.build(time_steps=24, num_features=21, reg=0.0005)
    opt = Adam(1e-3)
    model.compile(loss= 'mae', optimizer=opt, metrics=["mae", "mape"])
# otherwise, load the checkpoint from disk
else:
    print("[INFO] loading {}...".format(model))
    model = load_model(model)

    # update the learning rate
    print("[INFO] old learning rate: {}".format(K.get_value(model.optimizer.lr)))
    K.set_value(model.optimizer.lr, 1e-4)
    print("[INFO] new learning rate: {}".format(K.get_value(model.optimizer.lr)))

[INFO] loading C:\Users\musta\Desktop\ML lab\lab 8\E1-cp-0059-loss0.01.h5...
[INFO] old learning rate: 0.0010000000474974513
[INFO] new learning rate: 9.999999747378752e-05


In [31]:
epochs = 10
verbose = 1 #0
batch_size = 32
History = model.fit(train_X,
                        train_y,
                        batch_size=batch_size,   
                        epochs = epochs, 
                        validation_data = (validation_X,validation_y),
                        callbacks=callbacks,
                        verbose = verbose)

Epoch 1/10
2653/2653 [==============================] - ETA: 0s - loss: 0.0064 - mae: 0.0064 - mape: 418.1795
Epoch 1: val_loss improved from inf to 0.00654, saving model to C:\Users\musta\Desktop\ML lab\lab 8\E2-cp-0001-loss0.01.h5
2653/2653 [==============================] - 39s 14ms/step - loss: 0.0064 - mae: 0.0064 - mape: 418.1795 - val_loss: 0.0065 - val_mae: 0.0065 - val_mape: 3.6013
Epoch 2/10
2648/2653 [============================>.] - ETA: 0s - loss: 0.0064 - mae: 0.0064 - mape: 403.7992
Epoch 2: val_loss improved from 0.00654 to 0.00642, saving model to C:\Users\musta\Desktop\ML lab\lab 8\E2-cp-0002-loss0.01.h5
2653/2653 [==============================] - 20s 7ms/step - loss: 0.0064 - mae: 0.0064 - mape: 403.1091 - val_loss: 0.0064 - val_mae: 0.0064 - val_mape: 3.2198
Epoch 3/10
2649/2653 [============================>.] - ETA: 0s - loss: 0.0063 - mae: 0.0063 - mape: 388.9308
Epoch 3: val_loss did not improve from 0.00642
2653/2653 [==============================] - 20s 8ms

In [33]:

model = load_model(r'C:\Users\syedm\Desktop\Electrical (Communication)\8TH SEMESTER 2025\INTRODUCTION TO MACHINE LEARNING LAB\MACHINE LAB WORKS\LAB 8\E2-cp-0007-loss0.01.h5')

y_pred_scaled   = model.predict(test_X)
y_pred          = scaler.inverse_transform(y_pred_scaled)
y_test_unscaled = scaler.inverse_transform(test_y)
# Mean Absolute Error (MAE)
MAE = np.mean(abs(y_pred - y_test_unscaled)) 
print('Mean Absolute Error (MAE): ' + str(np.round(MAE, 2)))

# Median Absolute Error (MedAE)
MEDAE = np.median(abs(y_pred - y_test_unscaled))
print('Median Absolute Error (MedAE): ' + str(np.round(MEDAE, 2)))

# Mean Squared Error (MSE)
MSE = np.square(np.subtract(y_pred, y_test_unscaled)).mean()
print('Mean Squared Error (MSE): ' + str(np.round(MSE, 2)))

# Root Mean Squarred Error (RMSE) 
RMSE = np.sqrt(np.mean(np.square(y_pred - y_test_unscaled)))
print('Root Mean Squared Error (RMSE): ' + str(np.round(RMSE, 2)))

# Mean Absolute Percentage Error (MAPE)
MAPE = np.mean((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Mean Absolute Percentage Error (MAPE): ' + str(np.round(MAPE, 2)) + ' %')

# Median Absolute Percentage Error (MDAPE)
MDAPE = np.median((np.abs(np.subtract(y_test_unscaled, y_pred)/ y_test_unscaled))) * 100
print('Median Absolute Percentage Error (MDAPE): ' + str(np.round(MDAPE, 2)) + ' %')

print('\n\ny_test_unscaled.shape= ',y_test_unscaled.shape)
print('y_pred.shape= ',y_pred.shape)

379/379 [==============================] - 8s 18ms/step
Mean Absolute Error (MAE): 101.27
Median Absolute Error (MedAE): 79.49
Mean Squared Error (MSE): 18059.37
Root Mean Squared Error (RMSE): 134.39
Mean Absolute Percentage Error (MAPE): 0.69 %
Median Absolute Percentage Error (MDAPE): 0.55 %


y_test_unscaled.shape=  (12105, 1)
y_pred.shape=  (12105, 1)


# lab report 

## Lab 1